# **Imports & Iperparameters**

In [ ]:
import os
import cv2
import tf_keras
import pathlib
import patoolib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import main.utilities.utils as us
import main.models.II_VDSR.architectures.architectures_vdsr as arcs

from tqdm import tqdm
from glob import glob
from tf_keras.layers import *
from tf_keras.optimizers import *
from tf_keras.utils import plot_model
from tf_keras.preprocessing.image import *
from tf_keras.preprocessing import  image_dataset_from_directory

**Iperparametri** \
Questi sono gli Iperparametri che possiamo decidere noi

In [ ]:
# SuperRes
BATCH_SIZE  = 256
SCALE       = 2.0

# DataSet
INPUT_DIM   = 32
LABEL_SIZE  = 64
STRIDE      = 14
PAD         = 0

# Model
EPOCHS = 10

# Random Seed
SEED        = 42
tf.random.set_seed(SEED)

# **Load Data**

In [ ]:
# Come prima cosa definiamo il percorso base di questo file
current_path = (os.path.dirname(os.path.realpath(os.getcwd())) + "\\" + os.path.basename(os.getcwd()))
current_path

In [ ]:
# Estraiamo i Dati dall'archivio nella cartella ".\data\VDSR_Dataset_TrainTest"
data_zip_path = current_path + "\images\VDSR_Dataset_TrainTest.rar"
data_imgs_path = patoolib.extract_archive(data_zip_path, outdir=(current_path + "\\images"))

In [ ]:
# Definiamo una lista contenente i percorsi di ogni immagine

# Train
train_data_imgs_path = (current_path + "\\images\\train_data\\291")
file_train_pattern = (pathlib.Path(train_data_imgs_path) / "*")
file_train_pattern = str(file_train_pattern)
dataset_train_paths = [*glob(file_train_pattern)]
print("number of images:", len(dataset_train_paths))

**Visualize the Data**

In [ ]:
img = plt.imread(np.random.choice(dataset_train_paths))
plt.imshow(img)
plt.axis("off")
print(img_to_array(img).shape)

# **Create Augmented Data**

In [ ]:
# Definiamo i percorsi delle Immagini che formeranno il Dataset
train_dir_lr = current_path + "\\data\\train\\lr"
train_dir_hr = current_path + "\\data\\train\\hr"
train_dir_lr, train_dir_hr

In [ ]:
# TRAIN
for image_path in tqdm(dataset_train_paths):
  filename = pathlib.Path(image_path).stem
  image = load_img(image_path)
  image = img_to_array(image)
  image = image.astype(np.uint8)
  image = us.tight_crop_image(image, SCALE)

  height, width = image.shape[:2]
  for y in range(0, height - INPUT_DIM + 1, STRIDE):
    for x in range(0, width - INPUT_DIM + 1, STRIDE):
      output = us.crop_output(image, x, y, PAD, LABEL_SIZE)  # High Res
      input = us.resize_image(output, 1.0/SCALE)             # Low Res
      cv2.imwrite(f"/content/data/train/lr/{filename}_{x}_{y}_input.png", cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
      cv2.imwrite(f"/content/data/train/hr/{filename}_{x}_{y}_output.png", cv2.cvtColor(output, cv2.COLOR_BGR2RGB))

# **Create Dataset**

In [ ]:
# Train Set
train_data_lr = image_dataset_from_directory(
    directory=train_dir_lr,
    labels=None,
    label_mode=None,
    image_size=(INPUT_DIM, INPUT_DIM),
    interpolation="bilinear",
    batch_size=BATCH_SIZE,
    seed=42
)

train_data_hr = image_dataset_from_directory(
    directory=train_dir_hr,
    labels=None,
    label_mode=None,
    image_size=(LABEL_SIZE, LABEL_SIZE),
    interpolation="bilinear",
    batch_size=BATCH_SIZE,
    seed=42
)

ds_train = tf.data.Dataset.zip(train_data_lr, train_data_hr)

**Visualize the Data**

In [ ]:
# Define the iterators
i_train = ds_train.as_numpy_iterator();

In [ ]:
# Plot Train Sample
sample_train = i_train.next()
us.plot_images([tf.cast(sample_train[1][255], tf.uint8), tf.cast(sample_train[0][255], tf.uint8)]) # HR, LR

# **Preprocess Data**

Qui mettiamo tutte le operazioni di perprocessamento dei Dati.\
Quando vorremo addestrare la rete potremo decidere quale di queste effettuare e quale no.

In [ ]:
# Normalize
ds_train = ds_train.map(us.normalizer)

# **Model**

**1. Create the Model**

In [ ]:
# Choose which model to use
model = arcs.VDSR_Vanilla((INPUT_DIM, INPUT_DIM, 3), 10, LABEL_SIZE=LABEL_SIZE)
model._name = ("VDSR_Vanilla_fit10")

**2. Compile the Model**

In [ ]:
# Scegliamo quale Loss e quale Optimizer assegnare al Modello

my_loss = tf_keras.losses.mean_squared_error;
my_opt = tf_keras.optimizers.Adam(learning_rate=0.0001);

model.compile(
  loss=my_loss,
  optimizer=my_opt,
  metrics=[us.PSNR_metric, us.SSIM_metric]
)

In [ ]:
# Eseguiamo il Plot del Modello
model.summary()
plot_model(model, show_shapes=True, rankdir="LR")

**3. Fit the Model**

In [ ]:
# Define TensorBoard Callbacks
tb_callback = tf_keras.callbacks.TensorBoard(log_dir=("logs/" + str(model._name)), histogram_freq=1)

In [ ]:
# Fit
model.fit(ds_train, epochs=EPOCHS, callbacks=[tb_callback])

# **References**

[1] Review: VDSR (Super Resolution) - "https://towardsdatascience.com/review-vdsr-super-resolution-f8050d49362f"
[2] Accurate Image Super-Resolution Using Very Deep Convolutional Networks - "https://franciscofarinha.ca/post/vdsr_paper/"
[3] VDSR-Keras - "https://github.com/GeorgeSeif/VDSR-Keras/blob/master/test_vdsr.py"